In [29]:
import numpy as np

import tensorflow.random as tfr
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl

import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

seed = 42
input_img_size = 96

np.random.seed(seed)
tfr.set_seed(seed)

In [30]:
filtered_dataset_path = '../Preprocessing/augmented_dataset.npz'
data = np.load(filtered_dataset_path)
X_train = data['images_t']
y_train = data['labels_t']
X_val = data['images_v']
y_val = data['labels_v']

# Define a mapping of labels to their corresponding digit names
labels = {0:'Basophil', 1:'Eosinophil', 2:'Erythroblast', 3:'Immature granulocytes', 4:'Lymphocyte', 5:'Monocyte', 6:'Neutrophil', 7:'Platelet'}

# Save unique labels
unique_labels = list(labels.values())

In [31]:
# Import keras
import keras_cv as kcv

In [32]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,         # Equivalent to RandomRotation
    width_shift_range=0.2,     # Equivalent to RandomTranslation on x-axis
    height_shift_range=0.2,    # Equivalent to RandomTranslation on y-axis
    zoom_range=0.2,            # Equivalent to RandomZoom
    horizontal_flip=True,      # Equivalent to RandomFlip (horizontal)
    vertical_flip=True         # Equivalent to RandomFlip (vertical)
)

train_generator = datagen.flow(X_train, y_train)

In [33]:
# Re-load the model after transfer learning
# ft_model = tfk.models.load_model('weights.keras')
model_filename = 'TL_dense_59.57.keras'
ft_model = tfk.models.load_model(model_filename)

# Display a summary of the model architecture
# ft_model.summary(expand_nested=True, show_trainable=True)
#! pip install pydot
# Display model architecture with layer shapes and trainable parameters
#tfk.utils.plot_model(ft_model, expand_nested=True, show_trainable=True, show_shapes=True, dpi=70)

In [34]:
print("weights:", len(ft_model.weights))
print("trainable_weights:", len(ft_model.trainable_weights))
print("non_trainable_weights:", len(ft_model.non_trainable_weights))

weights: 606
trainable_weights: 2
non_trainable_weights: 604


In [35]:
ft_model.get_layer('densenet121').trainable = True

# Set all MobileNetV3Small layers as non-trainable
for layer in ft_model.get_layer('densenet121').layers:
    layer.trainable = False
print(len(ft_model.get_layer('densenet121').layers))
# Enable training only for Conv2D layers
for i, layer in enumerate(ft_model.get_layer('densenet121').layers):
    if isinstance(layer, tfk.layers.Conv2D):
        layer.trainable = True
        print(i, layer.name, type(layer).__name__, layer.trainable)

427
2 conv1_conv Conv2D True
9 conv2_block1_1_conv Conv2D True
12 conv2_block1_2_conv Conv2D True
16 conv2_block2_1_conv Conv2D True
19 conv2_block2_2_conv Conv2D True
23 conv2_block3_1_conv Conv2D True
26 conv2_block3_2_conv Conv2D True
30 conv2_block4_1_conv Conv2D True
33 conv2_block4_2_conv Conv2D True
37 conv2_block5_1_conv Conv2D True
40 conv2_block5_2_conv Conv2D True
44 conv2_block6_1_conv Conv2D True
47 conv2_block6_2_conv Conv2D True
51 pool2_conv Conv2D True
55 conv3_block1_1_conv Conv2D True
58 conv3_block1_2_conv Conv2D True
62 conv3_block2_1_conv Conv2D True
65 conv3_block2_2_conv Conv2D True
69 conv3_block3_1_conv Conv2D True
72 conv3_block3_2_conv Conv2D True
76 conv3_block4_1_conv Conv2D True
79 conv3_block4_2_conv Conv2D True
83 conv3_block5_1_conv Conv2D True
86 conv3_block5_2_conv Conv2D True
90 conv3_block6_1_conv Conv2D True
93 conv3_block6_2_conv Conv2D True
97 conv3_block7_1_conv Conv2D True
100 conv3_block7_2_conv Conv2D True
104 conv3_block8_1_conv Conv2D True

In [36]:
# Set the number of layers to freeze
N = 200

# Set the first N layers as non-trainable
for i, layer in enumerate(ft_model.get_layer('densenet121').layers[:N]):
    layer.trainable = False

# Print layer indices, names, and trainability status
for i, layer in enumerate(ft_model.get_layer('densenet121').layers):
    print(i, layer.name, layer.trainable)

# Display a summary of the model architecture
#ft_model.summary(expand_nested=True)

# Display model architecture with layer shapes and trainable parameters
tfk.utils.plot_model(ft_model, expand_nested=True, show_trainable=True, show_shapes=True, dpi=70)

0 input_layer False
1 zero_padding2d False
2 conv1_conv False
3 conv1_bn False
4 conv1_relu False
5 zero_padding2d_1 False
6 pool1 False
7 conv2_block1_0_bn False
8 conv2_block1_0_relu False
9 conv2_block1_1_conv False
10 conv2_block1_1_bn False
11 conv2_block1_1_relu False
12 conv2_block1_2_conv False
13 conv2_block1_concat False
14 conv2_block2_0_bn False
15 conv2_block2_0_relu False
16 conv2_block2_1_conv False
17 conv2_block2_1_bn False
18 conv2_block2_1_relu False
19 conv2_block2_2_conv False
20 conv2_block2_concat False
21 conv2_block3_0_bn False
22 conv2_block3_0_relu False
23 conv2_block3_1_conv False
24 conv2_block3_1_bn False
25 conv2_block3_1_relu False
26 conv2_block3_2_conv False
27 conv2_block3_concat False
28 conv2_block4_0_bn False
29 conv2_block4_0_relu False
30 conv2_block4_1_conv False
31 conv2_block4_1_bn False
32 conv2_block4_1_relu False
33 conv2_block4_2_conv False
34 conv2_block4_concat False
35 conv2_block5_0_bn False
36 conv2_block5_0_relu False
37 conv2_block

In [37]:
# for i, layer in enumerate(ft_model.get_layer('densenet121').layers):
#     if isinstance(layer, tfk.layers.BatchNormalization):
#         layer.trainable = True
#         print(i, layer.name, type(layer).__name__, layer.trainable)

In [38]:

# Imposta i learning rates specifici per ciascun tipo di layer trainabile
# layer_lr_dict = {
#     tfk.layers.Conv2D: 1e-5,
#     tfk.layers.BatchNormalization: 1e-3,
# }

In [39]:
# from tensorflow.keras.callbacks import Callback

# class CustomLearningRateScheduler(Callback):
#     def __init__(self, layer_lr_dict):
#         super(CustomLearningRateScheduler, self).__init__()
#         self.layer_lr_dict = layer_lr_dict

#     def on_train_batch_begin(self, batch, logs=None):
#         print(type(self.model.optimizer.learning_rate))
#         for layer in self.model.get_layer('densenet121').layers:
#             if layer.trainable and layer.trainable_weights:
#                 for layer_type, lr in self.layer_lr_dict.items():
#                     if isinstance(layer, layer_type):
#                         for weight in layer.trainable_weights:
#                             tfk.backend.set_value(self.model.optimizer.learning_rate, lr)
#                             break  # Applica il learning rate solo una volta per layer


In [ ]:
# Compile the model
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.AdamW(learning_rate=5e-5,weight_decay = 1e-8), metrics=['accuracy'])# lr_scheduler = CustomLearningRateScheduler(layer_lr_dict)

In [41]:
# Fine-tune the model
ft_history = ft_model.fit(
    train_generator, 
    batch_size = 16,
    epochs = 10,
    validation_data = (X_val, y_val),
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=2, restore_best_weights=True)]
).history

# Calculate and print the final validation accuracy
final_val_accuracy = round(max(ft_history['val_accuracy'])* 100, 2)
print(f'Final validation accuracy: {final_val_accuracy}%')

# Save the trained model to a file with the accuracy included in the filename
model_filename = 'weights.keras'
ft_model.save(model_filename)

# Delete the model to free up resources
del ft_model

Epoch 1/10


d:\ANN\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1224/1224 ━━━━━━━━━━━━━━━━━━━━ 506s 396ms/step - accuracy: 0.8225 - loss: 0.5049 - val_accuracy: 0.7707 - val_loss: 0.7656
Epoch 2/10
1224/1224 ━━━━━━━━━━━━━━━━━━━━ 470s 384ms/step - accuracy: 0.9074 - loss: 0.2617 - val_accuracy: 0.7886 - val_loss: 0.7226
Epoch 3/10
1224/1224 ━━━━━━━━━━━━━━━━━━━━ 476s 389ms/step - accuracy: 0.9252 - loss: 0.2143 - val_accuracy: 0.8015 - val_loss: 0.6361
Epoch 4/10
1224/1224 ━━━━━━━━━━━━━━━━━━━━ 472s 386ms/step - accuracy: 0.9368 - loss: 0.1824 - val_accuracy: 0.8197 - val_loss: 0.5848
Epoch 5/10
1224/1224 ━━━━━━━━━━━━━━━━━━━━ 472s 386ms/step - accuracy: 0.9449 - loss: 0.1583 - val_accuracy: 0.8289 - val_loss: 0.5813
Epoch 6/10
1224/1224 ━━━━━━━━━━━━━━━━━━━━ 458s 374ms/step - accuracy: 0.9476 - loss: 0.1476 - val_accuracy: 0.8051 - val_loss: 0.6605
Epoch 7/10
1224/1224 ━━━━━━━━━━━━━━━━━━━━ 459s 375ms/step - accuracy: 0.9530 - loss: 0.1318 - val_accuracy: 0.8140 - val_loss: 0.6091
Final validation accuracy: 82.89%


In [ ]:
## Train and save
# Load the saved model
model = tfk.models.load_model(model_filename)

# Compute the confusion matrix
predictions = model.predict(X_val)
pred_classes = np.argmax(predictions, axis=-1)
true_classes = np.argmax(y_val, axis=-1)
cm = confusion_matrix(true_classes, pred_classes)

# Plot the confusion matrix
plt.figure(figsize=(10,8))
sns.heatmap(cm.T, annot=True, fmt='', xticklabels=unique_labels, yticklabels=unique_labels, cmap='Blues')
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

156/175 ━━━━━━━━━━━━━━━━━━━━ 5s 286ms/step

In [ ]:
%%writefile model.py
import numpy as np

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl


class Model:
    def __init__(self):
        """
        Initialize the internal state of the model. Note that the __init__
        method cannot accept any arguments.

        The following is an example loading the weights of a pre-trained
        model.
        """
        self.neural_network = tfk.models.load_model('weights.keras')

    def predict(self, X):
        """
        Predict the labels corresponding to the input X. Note that X is a numpy
        array of shape (n_samples, 96, 96, 3) and the output should be a numpy
        array of shape (n_samples,). Therefore, outputs must no be one-hot
        encoded.

        The following is an example of a prediction from the pre-trained model
        loaded in the __init__ method.
        """
        X = X / 255.0
        preds = self.neural_network.predict(X)
        if len(preds.shape) == 2:
            preds = np.argmax(preds, axis=1)
        return preds

Overwriting model.py
